Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

/home/joelpiet/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Read and clean the data

In [2]:
data = pd.read_csv('data/train.csv')
display(data.head())

# parentspecies column is categorical, so we need to convert it to numerical
data['parentspecies'] = data['parentspecies'].astype('category')
data['parentspecies'] = data['parentspecies'].cat.codes

# make psat_pa log base 10
data['pSat_Pa'] = np.log10(data['pSat_Pa'])

display(data.head())

# check for missing values
display(data.isnull().sum())

,Id,MW,NumOfAtoms,NumOfC,NumOfO,NumOfN,NumHBondDonors,NumOfConf,NumOfConfUsed,parentspecies,...,ether..alicyclic.,nitrate,nitro,aromatic.hydroxyl,carbonylperoxynitrate,peroxide,hydroperoxide,carbonylperoxyacid,nitroester,pSat_Pa
0,0,30.010565,4,1,1,0,0,1,1,apin_decane_toluene,...,0,0,0,0,0,0,0,0,0,641974.491
1,1,74.995643,6,1,3,1,0,1,1,toluene,...,0,0,1,0,0,0,0,0,0,10295.712
2,2,102.990558,8,2,4,1,0,2,2,toluene,...,0,0,1,0,0,0,0,0,0,13517.575
3,3,118.985472,9,2,5,1,1,3,1,toluene,...,0,0,1,0,0,0,0,0,0,241.630
4,4,134.980387,10,2,6,1,1,3,3,toluene,...,0,0,1,0,0,0,0,1,0,315.357


,Id,MW,NumOfAtoms,NumOfC,NumOfO,NumOfN,NumHBondDonors,NumOfConf,NumOfConfUsed,parentspecies,...,ether..alicyclic.,nitrate,nitro,aromatic.hydroxyl,carbonylperoxynitrate,peroxide,hydroperoxide,carbonylperoxyacid,nitroester,pSat_Pa
0,0,30.010565,4,1,1,0,0,1,1,2,...,0,0,0,0,0,0,0,0,0,5.807518
1,1,74.995643,6,1,3,1,0,1,1,6,...,0,0,1,0,0,0,0,0,0,4.012656
2,2,102.990558,8,2,4,1,0,2,2,6,...,0,0,1,0,0,0,0,0,0,4.130899
3,3,118.985472,9,2,5,1,1,3,1,6,...,0,0,1,0,0,0,0,0,0,2.383151
4,4,134.980387,10,2,6,1,1,3,3,6,...,0,0,1,0,0,0,0,1,0,2.498802


Id                              0
MW                              0
NumOfAtoms                      0
NumOfC                          0
NumOfO                          0
NumOfN                          0
NumHBondDonors                  0
NumOfConf                       0
NumOfConfUsed                   0
parentspecies                   0
C.C..non.aromatic.              0
C.C.C.O.in.non.aromatic.ring    0
hydroxyl..alkyl.                0
aldehyde                        0
ketone                          0
carboxylic.acid                 0
ester                           0
ether..alicyclic.               0
nitrate                         0
nitro                           0
aromatic.hydroxyl               0
carbonylperoxynitrate           0
peroxide                        0
hydroperoxide                   0
carbonylperoxyacid              0
nitroester                      0
pSat_Pa                         0
dtype: int64

fit the model

In [34]:
#fit xgb model target is pSat_Pa
target = data['pSat_Pa']
features = data.drop(['pSat_Pa'], axis=1)
features = features.drop(['Id'], axis=1)

#features = features.drop(['nitro'], axis=1)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.05, random_state=42)


model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.01, n_estimators=1000)

model.fit(X_train, y_train)

# make predictions
preds = model.predict(X_test)

# calculate rmse
rmse = mean_squared_error(y_test, preds, squared=False)

print("RMSE: %f" % (rmse))

# r2 score
r2 = r2_score(y_test, preds)

print("R2: %f" % (r2))

#fit random forest model target is pSat_Pa

model2 = RandomForestRegressor()

model2.fit(X_train, y_train)

# make predictions
preds2 = model2.predict(X_test)

# calculate rmse
rmse2 = mean_squared_error(y_test, preds2, squared=False)

print("RMSE: %f" % (rmse2))

# r2 score
r2_2 = r2_score(y_test, preds2)

print("R2: %f" % (r2_2))

#fit hist gradient boosting model target is pSat_Pa

model3 = HistGradientBoostingRegressor(loss='squared_error', learning_rate=0.07, max_iter=100)

model3.fit(X_train, y_train)

# make predictions

preds3 = model3.predict(X_test)

# calculate rmse
rmse3 = mean_squared_error(y_test, preds3, squared=False)

print("RMSE: %f" % (rmse3))

# r2 score
r2_3 = r2_score(y_test, preds3)

print("R2: %f" % (r2_3))


RMSE: 1.116113
R2: 0.738300
RMSE: 1.170051
R2: 0.712394
RMSE: 1.114813
R2: 0.738909


Lets try different number of features

In [32]:
# at each iteration add feature which improves r2 score the most using backward selection and xbgoost

target = data['pSat_Pa']
features = data.drop(['pSat_Pa'], axis=1)
features = features.drop(['Id'], axis=1)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.05, random_state=42)

best_features = []

for i in range(0, len(features.columns) - 1):

    # find feature which improves r2 score the most
    r2_list = []
    for feature in X_train.columns:
        X_train_temp = X_train.copy()
        X_train_temp = X_train_temp.drop([feature], axis=1)
        X_test_temp = X_test.copy()
        X_test_temp = X_test_temp.drop([feature], axis=1)

        model = HistGradientBoostingRegressor(loss='squared_error', learning_rate=0.07, max_iter=100)

        model.fit(X_train_temp, y_train)

        # make predictions
        preds = model.predict(X_test_temp)

        # calculate rmse
        rmse = mean_squared_error(y_test, preds, squared=False)

        # r2 score
        r2 = r2_score(y_test, preds)

        r2_list.append(r2)

    # find feature which reduced r2 score the most
    r2_list = np.array(r2_list)
    best_feature = X_train.columns[np.argmin(r2_list)]

    # add best feature to X_train and X_test
    X_train = X_train.drop([best_feature], axis=1)
    X_test = X_test.drop([best_feature], axis=1)

    # add best feature to best_features
    best_features.append(best_feature)

print(best_features)

['NumHBondDonors', 'hydroxyl..alkyl.', 'NumOfConf', 'ketone', 'carboxylic.acid', 'NumOfAtoms', 'MW', 'NumOfO', 'NumOfC', 'hydroperoxide', 'NumOfConfUsed', 'aldehyde', 'carbonylperoxyacid', 'ether..alicyclic.', 'ester', 'parentspecies', 'nitro', 'carbonylperoxynitrate', 'NumOfN', 'nitroester', 'aromatic.hydroxyl', 'C.C..non.aromatic.', 'C.C.C.O.in.non.aromatic.ring', 'peroxide']


In [45]:
# at each iteration add feature which improves r2 score the most using forward selection and xbgoost

target = data['pSat_Pa']
features = data.drop(['pSat_Pa'], axis=1)
features = features.drop(['Id'], axis=1)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.05, random_state=42)

best_features = []

X_train_empty = pd.DataFrame()
X_test_empty = pd.DataFrame()

for i in range(0, len(features.columns) - 1):
    
        # find feature which improves r2 score the most
        r2_list = []
        for feature in X_train.columns:
            X_train_temp = X_train_empty.copy()
            X_train_temp[feature] = X_train[feature]
            X_test_temp = X_test_empty.copy()
            X_test_temp[feature] = X_test[feature]
    
            model = HistGradientBoostingRegressor(loss='squared_error', learning_rate=0.07, max_iter=100)

            model.fit(X_train_temp, y_train)

            # make predictions
            preds = model.predict(X_test_temp)
    
            # calculate rmse
            rmse = mean_squared_error(y_test, preds, squared=False)
    
            # r2 score
            r2 = r2_score(y_test, preds)
    
            r2_list.append(r2)
    
        # find feature which improves r2 score the most
        r2_list = np.array(r2_list)
        best_feature = X_train.columns[np.argmax(r2_list)]
    
        X_train_empty[best_feature] = X_train[best_feature]
        X_test_empty[best_feature] = X_test[best_feature]

        # add best feature to X_train and X_test
        X_train = X_train.drop([best_feature], axis=1)
        X_test = X_test.drop([best_feature], axis=1)
    
        # add best feature to best_features
        best_features.append(best_feature)

print(best_features)


['NumOfAtoms', 'carbonylperoxynitrate', 'parentspecies', 'carboxylic.acid', 'carbonylperoxyacid', 'C.C..non.aromatic.', 'C.C.C.O.in.non.aromatic.ring', 'nitroester', 'ester', 'hydroperoxide', 'aromatic.hydroxyl', 'aldehyde', 'peroxide', 'ether..alicyclic.', 'nitrate', 'ketone', 'NumOfO', 'NumOfN', 'NumHBondDonors', 'nitro', 'hydroxyl..alkyl.', 'MW', 'NumOfConfUsed', 'NumOfC']


In [33]:
# according to best_features list fit xgb model on 1, 2, ... n features and find best cv score

target = data['pSat_Pa']
features = data.drop(['pSat_Pa'], axis=1)
features = features.drop(['Id'], axis=1)

models = []
cross_val_scores = []

for i in range(1, len(best_features)):
    temp_features = features[best_features[0:i]]
    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(temp_features, target, test_size=0.05, random_state=42)
    #cross validation
    #cv = KFold(n_splits=5)
    m = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.01, n_estimators=1000)
    #scores = cross_val_score(m, temp_features, target, cv=cv)
    models.append(m)

    m.fit(X_train, y_train)

    # make predictions
    preds = m.predict(X_test)

    # calculate r2
    score = r2_score(y_test, preds)
    
    cross_val_scores.append(score)

print(cross_val_scores)

[0.48243232511826606, 0.5029840296082781, 0.5706559739612401, 0.5809747102985436, 0.5854838340925044, 0.6390333863878561, 0.6851865631404872, 0.6888180914293847, 0.7010464362462038, 0.7030961237172522, 0.7091594353576587, 0.7119854966660586, 0.7114646461677966, 0.7146456570117171, 0.7194202297211093, 0.7221238507217518, 0.7243536922871514, 0.7264573266046666, 0.7265901666124344, 0.7272098673119822, 0.7264751639454557, 0.7287969743138816, 0.7295836008032368]


Make predictions on test data

In [36]:
test_data = pd.read_csv('data/test.csv')

# parentspecies column is categorical, so we need to convert it to numerical
test_data['parentspecies'] = test_data['parentspecies'].astype('category')
test_data['parentspecies'] = test_data['parentspecies'].cat.codes

# remove Id column
test_data_noID = test_data.drop(['Id'], axis=1)

test_data_noID = test_data_noID[best_features[0:-8]]

preds = models[-8].predict(test_data_noID)

# create submission file
submission = pd.DataFrame({'Id': test_data['Id'], 'target': preds})

submission.to_csv('submission.csv', index=False)